In [26]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data}, grad={self.grad})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [28]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot.render(directory='graphs')

In [29]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]
  
  def zero_grad(self):
    for p in self.parameters():
      p.grad = 0.0
  
  def update(self, lr=0.1):
    for p in self.parameters():
      p.data = -lr * p.grad
          

In [30]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
print(n.parameters())
n(x)

[Value(data=-0.6866307737236155, grad=0.0), Value(data=-0.7452110592515695, grad=0.0), Value(data=-0.1461124180793787, grad=0.0), Value(data=-0.42758917010076747, grad=0.0), Value(data=-0.1539533657714578, grad=0.0), Value(data=-0.7915388137601387, grad=0.0), Value(data=-0.1692676218233169, grad=0.0), Value(data=0.3396536630220943, grad=0.0), Value(data=0.037571696625607665, grad=0.0), Value(data=-0.9700774499092877, grad=0.0), Value(data=-0.4575134233000955, grad=0.0), Value(data=0.8623572275882625, grad=0.0), Value(data=0.37768544754096034, grad=0.0), Value(data=0.20628314682541982, grad=0.0), Value(data=-0.15415933182689723, grad=0.0), Value(data=0.796658837651059, grad=0.0), Value(data=0.536116832176617, grad=0.0), Value(data=0.5611505407356372, grad=0.0), Value(data=-0.49266988194042693, grad=0.0), Value(data=0.5956162422132556, grad=0.0), Value(data=0.9122929103430713, grad=0.0), Value(data=-0.37292695389273356, grad=0.0), Value(data=-0.1348546810901885, grad=0.0), Value(data=0.2

Value(data=0.6480805947186898, grad=0.0)

In [31]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired target

In [32]:
for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  n.zero_grad()
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)

0 5.458779523225546
1 4.664210224292212
2 5.183198218430247
3 4.386930984951293
4 4.898069926226806
5 3.299859704484054
6 4.086224244781954
7 1.1619808027662033
8 1.0065499774422133
9 1.3093880089024814
10 0.17153060440176224
11 0.06617245484342965
12 0.05628162056683992
13 0.049838841407583344
14 0.044995680713417786
15 0.04111198558813155
16 0.03788550764546951
17 0.035144291645663996
18 0.03277824625849876
19 0.03071127514098696


In [33]:
draw_dot(loss)

'graphs\\Digraph.gv.svg'